In [1]:
import Robogame as rg
import networkx as nx
import altair as alt
import time, json
import pandas as pd
import numpy as np

# Post-Match Analysis

This is an example of postgame analysis. We're going to read the real data (the csv/network/whatever you want) and the log file from the actual match.

In [2]:
# load the real game data
robotdata = pd.read_csv("../server/example1/examplematch1.robotdata.csv")

# let's load up the log file of the match
gamedata = {}
with open('../server/example1/bobvalice.json') as json_file:
    gamedata = json.load(json_file)

# Reading the gamedata object

```betlog``` is an array: It consists of the bets made by both teams. Objects in this array look like: ```{'time': 0.08, 'betby': 2, 'beton': 3, 'value': 56}``` meaning that at time .08 (fraction are ok here), team 2 guessed 56 for robot 3. If a team makes multiple bets at different times, you will see an entry for each bet (For example, you might see ```{'time': 5.08, 'betby': 2, 'beton': 3, 'value': 72}``` which means that team updated their bet on robot 3 to 72.

```team1_bets``` and ```team2_bets``` are arrays reflecting the final guess of each team. For example, if you say: ```gamedata['team2_bets'][45]``` this will give you the value of the that team 2 guessed for robot 45.

```winreasons``` is an array of which team won the robot and for what reason. For example, you can ask who won robot 42 by saying ```gamedata['winreasons'][42]```. The objects in this array look like this: ```{'winner':2,'reason':-1}```  The winner can either be -1 (no one), 1 (team 1), or 2 (team 2). A code of -1 means that one of the teams won because the other team put in a -1 (e.g., winner:2, reason:-1 means team 2 gets the bot because team 1, put in a guess of -1, if the winner is -1 and reason is -1, no team wanted the bot). If the code is 2 or 2.5, the team won because they were closer in their guess (a 2.5 means they were equally close and we flipped a coin). If the code is 1, that means the team won by the social network strategy (a code of 1.5 means the the same network values and we flipped a coin). If you see a -2 for either winner or reason, you might have a partial game.

```team1_int_bots``` and ```team2_int_bots``` are arrays: they reflect the bots the hacker was watching for each team at different times. For example, ```gamedata['team1_int_bots'][2]``` will give you team1's interests at time 2. An empty array here (```[]```) means they were interested in all robots. If you get something like ```[4,8]``` that means they were interested in robots 4 and 8 at time 2. 

```team1_int_parts``` and ```team2_int_parts``` are arrays: they reflect the parts the hacker was watching for each team at different times. For example, ```gamedata['team1_int_parts'][4]``` will give you team1's interests at time 4. An empty array here (```[]```) means they were interested in all robots. If you get something like ```[''Arakyd Vocabulator Model']``` that means they were interested in the 'Arakyd Vocabulator Model' part at time 4. 

```team1_hints_bots``` and ```team2_hints_bots``` are arrays: they relect the bot prediction hints the hacker prepared for the teams at each time. For example, ```gamedata['team1_hints_bots'][2]``` will give you the hints that were ready for team 1 at time 2. Hints are a series of objects that look like this: ```{'id': 86, 'time': 62, 'value': 56.450039697413786}``` which means that at time 62, robot 86 will output 56.45... 

```team1_hints_parts``` and ```team2_hints_parts``` are arrays: they relect the part prediction hints the hacker prepared for the teams at each time. For example, ```gamedata['team1_hints_parts'][2]``` will give you the hints that were ready for team 1 at time 2. Hints are a series of objects that look like this: ```{'id': 22, 'column': 'Arakyd Vocabulator Model', 'value': 'model A'}``` which means that at robot 22 has 'model A' for their 'Arakyd Vocabulator Model'  

In [3]:
# figure out what the correct answers for the guesses
correctAnswer = []

# figure out the correct answers
for row in robotdata[robotdata.id < 100].sort_values('id').iterrows():
    row = row[1]                 # the row data
    expires = int(row.expires)   # when the robot expired
    col = "t_"+str(expires)      # the column for that time
    valAtExp = row[col]          # the value at that time for that robot
    correctAnswer.append(valAtExp)

# Example analysis, let's compare team 1 and team 2's guesses to the real values

In [4]:
# first, we can look at the bets the two players made in relation to the true answer
# let's contrast the perfomance of the teams in relation to the true answer
dataset = pd.DataFrame({'id':np.arange(0,100),'realanswer': correctAnswer, 
                        'team1': gamedata['team1_bets'],'team2': gamedata['team2_bets']})

In [5]:
# find the difference between the realanswer and the teams
dataset['diff1'] = dataset['realanswer'] - dataset['team1']
dataset['diff2'] = dataset['realanswer'] - dataset['team2']

In [6]:
# make this into a longform array so we can see
longform = dataset.melt(id_vars='id',value_vars=['diff1','diff2'])

In [7]:
# plot the values
alt.Chart(longform).mark_line().encode(
    x='id:O',
    y='value:Q',
    color='variable:N'
)

alt.Chart(...)

# Example Analysis: Plot the reasons each team won by

In [13]:
# let's augment our winreasons data with the Productivity and Expiration
for i in np.arange(0,100):
    reason = gamedata['winreasons'][i]
    reason['prod'] = robotdata.at[i,'Productivity']
    reason['wintime'] = robotdata.at[i,'expires']

In [14]:
# make a dataframe
reasonframe = pd.read_json(json.dumps(gamedata['winreasons']))

In [15]:
reasonframe.sample(5)

,winner,reason,prod,wintime
52,2,2.0,34.430569,89
82,1,2.0,60.452266,79
88,2,1.0,91.734959,94
25,2,2.0,48.558440,43
54,2,1.0,43.614280,42


In [16]:
# what was the total productivity for each robot the teams won (based on reason)
alt.Chart(reasonframe).mark_bar().encode(
    x = 'winner:O',
    y = 'sum_prod:Q',
    color = 'reason:N'
).transform_aggregate(
    sum_prod='sum(prod)',
    groupby=["winner","reason"]
)

alt.Chart(...)

In [17]:
# what was the distribution of productivities
alt.Chart(reasonframe).mark_circle().encode(
    x = 'winner:O',
    y = 'prod:Q',
    color = 'reason:N'
)

alt.Chart(...)

In [36]:
# let's visualize our productivity gains or losses over time

# productivity over time
alt.Chart(reasonframe).mark_line().encode(
    x = 'wintime:Q',
    y = 'sum(prod)',
    color = 'winner:N'
)

alt.Chart(...)